In [7]:
!pip install google-auth google-api-python-client
!pip install networkx
!pip install openpyxl

     -------------------------------------- 250.0/250.0 kB 1.9 MB/s eta 0:00:00
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)


In [1]:
import os
import json
import requests
import numpy as np
import pandas as pd
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import networkx as nx
import matplotlib.pyplot as plt

# 設定ファイルのパス
config_file = 'config.json'

# 設定ファイルの読み込み
with open(config_file) as f:
    config = json.load(f)

# APIキーの取得
api_key = config['api_key']

# YouTube Data APIの設定
youtube = build('youtube', 'v3', developerKey=api_key)

In [21]:
# ビデオからチャンネルIDを取得
def get_channel_id(video_id):
    # 動画の詳細情報を取得
    video_response = youtube.videos().list(
        part='snippet',
        id=video_id
    ).execute()

    # 動画の詳細情報からチャンネルIDを取得
    channel_id = video_response['items'][0]['snippet']['channelId']

    return channel_id

# チャンネルのすべての動画のIDを取得
def get_video_id(channel_id):
    # チャンネル情報を取得
    channel_response = youtube.channels().list(
        part='snippet',
        id=channel_id
    ).execute()
    channel_name =     channel_response['items'][0]['snippet']['customUrl']

    # チャンネルの動画を取得
    videos = []
    next_page_token = None

    while True:
        # チャンネルのプレイリストを取得
        channel_response = youtube.channels().list(
            part='contentDetails',
            id=channel_id
        ).execute()

        # プレイリストのIDを取得
        uploads_playlist_id = channel_response['items'][0]['contentDetails']['relatedPlaylists']['uploads']

        # プレイリストの動画を取得
        playlist_items_response = youtube.playlistItems().list(
            part='snippet',
            playlistId=uploads_playlist_id,
            maxResults=50,
            pageToken=next_page_token
        ).execute()

        videos.extend(playlist_items_response['items'])

        next_page_token = playlist_items_response.get('nextPageToken')

        if not next_page_token:
            break

    # 結果をJSONファイルで保存
    df_videos = pd.DataFrame(videos)
    df_videos.to_json(f'data/videos/{channel_name}_videos.json', orient='records')


    # 取得した動画のIDを格納するリスト
    video_ids = []

    # 取得した動画のIDをリストに追加
    for video in videos:
        video_id = video['snippet']['resourceId']['videoId']
        video_ids.append(video_id)
    
    return video_ids, channel_name

# チャンネルのすべての動画からすべてのコメントを取得
def get_comments(video_ids, channel_name):
    # コメントを取得
    comments = []

    for video_id in video_ids:
        next_page_token = None

        while True:
            try:
                # 動画のコメントスレッドを取得
                comment_response = youtube.commentThreads().list(
                    part='snippet',
                    videoId=video_id,
                    maxResults=100,
                    pageToken=next_page_token
                ).execute()

                comments.extend(comment_response['items'])

                next_page_token = comment_response.get('nextPageToken')

                if not next_page_token:
                    break
            # コメントが無効になっている場合にエラーをはかないようにするため
            except HttpError as e:
                error = json.loads(e.content)
                if error['error']['errors'][0]['reason'] == 'commentsDisabled':
                    # コメントが無効になっている場合はスキップして次の動画に進む
                    break
                else:
                    # その他のエラーは処理を停止してエラーメッセージを表示
                    raise Exception(f"An error occurred: {error}")

    # 結果をJSONファイルで保存
    df_comments = pd.DataFrame(comments)
    df_comments.to_json(f'data/comments/{channel_name}_comments.json', orient='records')

    return comments

In [24]:
# チャンネルのIDからすべての動画を取得
# channel_id = "UCXjTiSGclQLVVU83GVrRM4w"
video_id = "I-PVrxKrOPA"
channel_id = get_channel_id(video_id) 
video_ids, channel_name = get_video_id(channel_id)

# 動画のリストからすべてのコメントを取得
comments = get_comments(video_ids, channel_name)

Exception: An error occurred: {'error': {'code': 403, 'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'errors': [{'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'domain': 'youtube.quota', 'reason': 'quotaExceeded'}]}}

In [28]:
df = pd.DataFrame(video_ids)
df.to_json(f'data/comments/{channel_name}_videoIds.json', orient='records')
print(channel_name)

@nktofficial


In [29]:
# 前回の制限から再開
def read_json(file):
    # JSONファイルを読み込む
    with open(file, 'r') as f:
        data = json.load(f)

    # DataFrameに変換する
    df = pd.DataFrame(data)
    return df

file_path = "data/comments/@nktofficial_videoIds.json"
video_ids = read_json(file_path)
channel_name = "video_ids"
comments = get_comments(video_ids, channel_name)

Exception: An error occurred: {'error': {'code': 404, 'message': 'The video identified by the <code><a href="/youtube/v3/docs/commentThreads/list#videoId">videoId</a></code> parameter could not be found.', 'errors': [{'message': 'The video identified by the <code><a href="/youtube/v3/docs/commentThreads/list#videoId">videoId</a></code> parameter could not be found.', 'domain': 'youtube.commentThread', 'reason': 'videoNotFound', 'location': 'videoId', 'locationType': 'parameter'}]}}